In [1]:
from dingo import HPolytope
import numpy as np
from dingo.gurobi_based_implementations import fast_inner_ball
from polytope_oracles.membership import Membership
from polytope import box2poly
from polytope import Polytope
from polytope_oracles.samplers import SphereBasedSampler
from dataclasses import dataclass, field
from typing import List, Iterable
from timeit import default_timer as timer
from collections import defaultdict
import pandas as pd
import pickle

In [2]:
@dataclass
class Query:
    point: np.ndarray
    inside: bool
    
    @property
    def get_point(self):
        return point


@dataclass
class PolytopeQuery:
    polytope: HPolytope
    queries: List[Query] = field(default_factory=list)
    
    @property
    def dimension(self):
        return polytope.dimension()
    
    def __getstate__(self):
        state = self.__dict__.copy()
        state['A'] = self.polytope.A()
        state['b'] = self.polytope.b()
        del state['polytope']
        return state
    
    def __setstate__(self, state):
        A = state.pop('A')
        b = state.pop('b')
        self.__dict__.update(state)
        self.polytope = HPolytope(A, b)

Define experimentation protocol

In [3]:
def is_in(point: np.ndarray, polytope: HPolytope, loop: bool = True):
    if loop:
        for i in range(polytope.A().shape[0]):
            h = polytope.A()[i, :]
            if h.dot(point) > polytope.b()[i]:
                return False
        return True
    else:
        return (polytope.A().dot(point) <= polytope.b()).all()

In [4]:
def is_in_polytope(point: np.ndarray, polytope):
    return point in polytope

In [5]:
from tqdm.auto import tqdm


def run_experiments(polytope_queries: Iterable[PolytopeQuery]):
    results = {"time": [], "success": []}

    for polytope_query in polytope_queries:
        n = polytope_query.polytope.A().shape[0]
        d = polytope_query.polytope.dimension()

        print(f'n={n} -- d={d}')

        tic = timer()
        membership = Membership(polytope=polytope_query.polytope)
        membership.initialize(m=16,max_m0=32,ef_construction=1)
        print(timer()-tic)
        
        for query in tqdm(polytope_query.queries):
            #tic = timer()
            #response = is_in(query.point, polytope_query.polytope, loop=True)
            #results['time'].append({"n": n, "d": d, "elapsed": timer() - tic, "type": "brute force loop"})
            #results['success'].append({"n": n, "d": d, "success": response == query.inside, "type": "brute force loop", "actual": query.inside})

            poly = Polytope(polytope_query.polytope.A(),polytope_query.polytope.b()) 
            tic = timer()
            #response = is_in(query.point, polytope_query.polytope, loop=False)
            response = is_in_polytope(query.point, poly)
            results['time'].append({"n": n, "d": d, "elapsed": timer() - tic, "type": "brute force vectorized"})
            results['success'].append({"n": n, "d": d, "success": response == query.inside, "type": "brute force vectorized", "actual": query.inside})

            tic = timer()
            response = membership.contains(query.point)
            results['time'].append({"n": n, "d": d, "elapsed": timer() - tic, "type": "membership"})
            results['success'].append({"n": n, "d": d, "success": response == query.inside, "type": "membership", "actual": query.inside})
    return pd.DataFrame(results['time']), pd.DataFrame(results['success'])

# Hyper-cube experiments

## Preparation

For a number of different dimensions, generate cubes of varying side sizes, so that we can measure the effect of absolute numbers in our method

In [6]:
side_sizes = [100000] # [1, 10, 100, 1000, 10000]
dimensions = [100, 1000]# [10, 50, 100, 200, 500, 1000, 2000, 5000, 10000]

cubes_by_side = defaultdict(list)
for side_size in side_sizes:
    for d in dimensions:
        box = box2poly([(- side_size / 2, side_size / 2)] * d)
        cubes_by_side[side_size].append(HPolytope(box.A, box.b))

Define hyper-cube sampling method. For each dimension we take a number of values uniformly distributes within the bounds of each cube.

In [7]:
def sample_cube(cube: HPolytope, n_points: int = 100, seed: int = 42):
    generator = np.random.default_rng()
    sides = np.multiply(cube.A(), cube.b()[:, np.newaxis])
    points = [list() for _ in range(n_points)]
    for side in range(sides.shape[1]):
        for i, value in enumerate(generator.uniform(np.min(sides[:, side]), np.max(sides[:, side]), n_points)):
            points[i].append(value)
    return [Query(p, True) for p in points]

In [8]:
polytope_queries_by_side = {}
for side in cubes_by_side:
    polytope_queries_by_side[side] = [PolytopeQuery(polytope, sample_cube(polytope)) for polytope in cubes_by_side[side]]

In [9]:
with open("./cube_queries.pickle", "wb") as fout:
    pickle.dump(polytope_queries_by_side, fout)

In [10]:
! ls -lh cube_queries.pickle

-rw-rw-r-- 1 vissarion vissarion 18M Φεβ  18 12:59 cube_queries.pickle


# Experiment

In [11]:
with open("./cube_queries.pickle", "rb") as fin:
    polytope_queries_by_side = pickle.load(fin)

In [12]:
results = []
for side, polytopes in polytope_queries_by_side.items():
    results.append(run_experiments(polytopes))

n=200 -- d=100
0.009789234987692907


  0%|          | 0/100 [00:00<?, ?it/s]

n=2000 -- d=1000
0.26756865199422464


  0%|          | 0/100 [00:00<?, ?it/s]

In [35]:
df_time = pd.concat([r[0] for r in results])
df_time_gb = df_time.groupby(["n", "d", "type"]).mean()
df_time_gb

elapsed
n    d    type                            
200  100  brute force vectorized  0.000024
          membership              0.000017
2000 1000 brute force vectorized  0.000957
          membership              0.000093

In [36]:
df_accuracy = pd.concat([r[1] for r in results])
df_accuracy_gb = df_accuracy.groupby(["n", "d", "type"]).sum()
df_accuracy_gb['success %'] = df_accuracy_gb["success"] / df_accuracy_gb["actual"] * 100
df_accuracy_gb

success  actual  success %
n    d    type                                              
200  100  brute force vectorized      100     100      100.0
          membership                   83     100       83.0
2000 1000 brute force vectorized      100     100      100.0
          membership                   12     100       12.0

# Random Polytopes

In [41]:
def random_polytope(d):
    n=2*d
    A = np.zeros((n,d))
    for j in range(n):
        normal = 0
        p = np.zeros((d))
        for i in range(d):
            p[i] = np.random.normal()
            normal += p[i] * p[i]
        normal = 1 / np.sqrt(normal);
        p *= normal;
        A[j,:] = p
    
    b = np.array([10.0]*n)
    P = HPolytope(A,b)
    return P

In [44]:
dimensions = [100, 1000]# [10, 50, 100, 200, 500, 1000, 2000, 5000, 10000]

polytopes = defaultdict(list)
for d in dimensions:
    polytopes[0].append(random_polytope(d))

In [45]:
def sample_polytope(P: HPolytope, n_points: int = 100, seed: int = 42):
    points = [list() for _ in range(n_points)]
    samples = P.generate_samples(walk_len = 5, number_of_points = n_points, 
                                 number_of_points_to_burn = 50, 
                                 boundary = True, cdhr = True, rdhr = False, 
                                 gaussian = False, set_L = False, billiard = False, 
                                 ball_walk = False, a = 0, L = 0)
    return [Query(p, True) for p in samples]


In [46]:
polytope_queries = {}
polytope_queries = [PolytopeQuery(polytope, sample_cube(polytope)) for polytope in polytopes[0]]

In [47]:
with open("./random_queries.pickle", "wb") as fout:
    pickle.dump(polytope_queries, fout)

In [48]:
! ls -lh random_queries.pickle

-rw-rw-r-- 1 vissarion vissarion 18M Φεβ   4 19:17 random_queries.pickle


In [49]:
with open("./random_queries.pickle", "rb") as fin:
    polytope_queries = pickle.load(fin)

In [50]:
results = []
for side, polytopes in polytope_queries.items():
    results.append(run_experiments(polytopes))

AttributeError: 'list' object has no attribute 'items'

# Old

In [13]:
d=100

In [14]:
box_out = box2poly([[-10001, 10001]] * d)
p_out = HPolytope(box_out.A, box_out.b)

In [15]:
m = Membership(polytope=p_out)

In [16]:
query = [0.01]*d

In [21]:
m.create_n2_nn(m=15)
m.contains_n2(query), m._index.search_by_vector(query, 1)

(False, [200])

In [9]:
m.create_hnsw_nn(m=5)
m._index.knn_query(query, k=1)

(array([[114]], dtype=uint64), array([[4.0008e+08]], dtype=float32))

(array([[9]], dtype=uint64), array([[4.0008e+08]], dtype=float32))

In [8]:
m.create_annoy_nn(5000)

TypeError: an integer is required (got type str)

In [39]:
m._index.search_by_vector([0.01]*d, 1, include_distances=True)

[(40, 0.8999999761581421)]

In [9]:
max_ball = fast_inner_ball(p.A(), p.b())
samples = p.generate_samples(walk_len=1, number_of_points=1000, radius=max_ball[1], inner_point=max_ball[0])

max_ball = fast_inner_ball(p_out.A(), p_out.b())
samples_out = p_out.generate_samples(walk_len=1, number_of_points=1000, boundary=True, radius=max_ball[1], inner_point=max_ball[0])

In [2]:
sampler = SphereBasedSampler()

In [3]:
d = 100

In [6]:
p = sampler.generate(d * 100, d)

In [ ]:
samples_out[0, :]

array([-1868.93666, -4586.7742 ,   514.71467,   576.38669,  3972.54032,
       -6774.19961, -3406.26128,  4121.20468,   528.16947,  1398.48769,
        -491.28975, -2680.04234, -2122.82087,  2996.4754 , -2513.2211 ,
       -1099.47762,  3685.66175,  -955.48015, -4364.11833,  4020.36754,
       -4421.60333,  4363.11061,  1480.50823, -5160.50437,    86.5002 ,
         326.42997, -4881.73193,   -94.8487 ,  3125.00804, -2779.26511,
       -3233.06436, -1533.41501, -2296.89104, -1038.61199,  2221.74466,
         114.91392,  2311.06249, -1522.20762,  -256.73503,  1350.9412 ,
        9144.16378,  1701.41608, -2791.53404, -4685.89362, -3543.41628,
       -4866.84575,  3819.95207, -2809.14988,  8556.10131, -1736.29506,
        5702.03686,  -647.9397 , -3966.69331,  2899.04392,  1127.25715,
        1588.8931 ,  1142.11892,  4142.42772, -2832.8605 , -1567.92027,
        4635.98342,   749.40934, -2319.70158,  1121.18358,  2014.69571,
        3987.81428, 10001.     ,  -703.74761, -3474.71281, -2777

In [ ]:
m = Membership(polytope=p)
m.create_annoy_nn(25000)

In [ ]:
from timeit import default_timer as timer

success = 0
in_c = 0

bf_t = 0
m_t = 0
for i in range(samples.shape[0]):
    tic = timer()
    inside = m.contains(samples[i, :])
    m_t += timer() - tic
    tic = timer()
    if is_in(p, samples[i, :]):
        in_c += 1
    bf_t += timer() - tic
#     print(f"{samples[i, :]} -- inside: {inside}")
    if inside:
        success += 1
success, in_c, m_t / samples.shape[0], bf_t / samples.shape[0], m_t, bf_t

(841,
 1000,
 0.01833995155838784,
 0.02667680139215372,
 18.33995155838784,
 26.67680139215372)

- memory leak sto acc. billiard walk gia n = 100000, d = 100
- kivoi se poli megales diastaseis xwris sampling sigrisi me b.f. (ws pros n, d)
- random polytopes me megalitero radius sti bala kai sigrisi me b.f. (ws pros n, d)
- recon polytopes

In [ ]:
success = 0
in_c = 0

bf_t = 0
m_t = 0

for i in range(samples_out.shape[0]):
    inside = m.contains(samples_out[i, :])
    if is_in(p, samples_out[i, :]):
        in_c += 1
#     print(f"{samples[i, :]} -- inside: {inside}")
    if not inside:
        success += 1
success, in_c

IndexError: Vector has wrong length (expected 500, got 1000)

In [ ]:
m = Membership(polytope=p_out)
m.create_annoy_nn(10000)

In [ ]:
success = 0
for i in range(samples.shape[0]):
    inside = m.contains(samples[i, :])
#     print(f"{samples[i, :]} -- inside: {inside}")
    if inside:
        success += 1
success

993